# Food API

In [ ]:
def get_ingredients_from_dish(dish_name):
    api_key = "your_spoonacular_api_key"
    url = f"https://api.spoonacular.com/recipes/complexSearch?query={dish_name}&number=1&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            recipe_id = data["results"][0]["id"]
            detail_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information?apiKey={api_key}"
            detail_response = requests.get(detail_url)
            if detail_response.status_code == 200:
                recipe_details = detail_response.json()
                return [ingredient["name"] for ingredient in recipe_details["extendedIngredients"]]
    return []

In [ ]:
import requests



def fetch_emission_factor(ingredient):
    """
    Fetch emission factor dynamically from a database or API.
    For simplicity, we mock a basic dynamic emission fetcher.
    """
    url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={ingredient}&search_simple=1&json=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        products = data.get("products", [])
        if products:
            # Retrieve category and use it to approximate emissions
            categories = products[0].get("categories", "").lower()
            if "meat" in categories:
                return 50  # Approximation for meats (kg CO2 per kg)
            elif "cheese" in categories:
                return 21  # Approximation for cheese
            elif "vegetables" in categories or "fruit" in categories:
                return 0.5  # Approximation for vegetables
            elif "pasta" in categories or "cereals" in categories:
                return 1.6  # Approximation for pasta/flour
    return 0  # Default emission if not found

def calculate_dish_emissions(dish_name, weights):
    """
    Calculate total emissions for a given dish based on its ingredients.
    """
    ingredients = get_ingredients_from_dish(dish_name)
    total_emissions = 0
    details = []

    for ingredient in ingredients:
        weight = weights.get(ingredient, 0)  # Default weight if not provided
        emission_factor = fetch_emission_factor(ingredient)
        emissions = emission_factor * weight
        total_emissions += emissions
        details.append({
            "ingredient": ingredient,
            "weight (kg)": weight,
            "emission_factor (kg CO2/kg)": emission_factor,
            "emissions (kg CO2)": emissions,
        })
    
    return total_emissions, details

def display_results(dish_name, total_emissions, details):
    """
    Display the CO₂ emissions breakdown for a dish.
    """
    print(f"\nCO₂ Emission Breakdown for {dish_name.title()}:")
    for item in details:
        print(f"- {item['ingredient']}: {item['emissions (kg CO2)']} kg CO₂ "
              f"(weight: {item['weight (kg)']} kg, emission factor: {item['emission_factor (kg CO2/kg)']} kg CO₂/kg)")
    print(f"\nTotal CO₂ emissions for {dish_name.title()}: {total_emissions:.2f} kg CO₂")

if __name__ == "__main__":
    # User-defined dish and weights (in kg)
    dish_name = input("Enter the name of the dish: ").strip()
    print(f"Fetching ingredients for {dish_name}...")

    # Prompt user for weights dynamically
    ingredients = get_ingredients_from_dish(dish_name)
    weights = {}
    for ingredient in ingredients:
        try:
            weight = float(input(f"Enter the weight of {ingredient} in kg (e.g., 0.2 for 200g): "))
            weights[ingredient] = weight
        except ValueError:
            print(f"Invalid weight for {ingredient}. Skipping.")

    # Calculate and display emissions
    if ingredients:
        total_emissions, details = calculate_dish_emissions(dish_name, weights)
        display_results(dish_name, total_emissions, details)
    else:
        print(f"No ingredients found for {dish_name}. Please check the dish name.")


Fetching ingredients for cheesburger...
No ingredients found for cheesburger. Please check the dish name.
